# Clone and Install Project
This step clones the coffee_level_detection repository and installs the package locally.

In [ ]:
!git clone https://github.com/PlayerPlanet/coffee_level_detection.git
%pip install ./coffee_level_detection

# Check Python Environment and Paths
This cell verifies the Python executable, checks that the training module is importable, and ensures dataset/image directories exist before running training.

In [ ]:
import sys, importlib.util, os
img_dir = ""
dataset_dir = ""
batch = 10
epochs = 100
executable_path = "coffee_level_detection.training.train_coffeeCNN"

# Basic checks
if not sys.executable:
    raise RuntimeError("No Python executable found in sys.executable")

spec = importlib.util.find_spec(executable_path)
if spec is None:
    print(f"Module '{executable_path}' not importable. Make sure package is installed or PYTHONPATH includes the project root.")
    # Optionally continue or raise
    raise SystemExit(1)

if dataset_dir and not os.path.exists(dataset_dir):
    print(f"Dataset directory does not exist: {dataset_dir}")
    raise SystemExit(1)

if img_dir and not os.path.exists(img_dir):
    print(f"Image directory does not exist: {img_dir}")
    raise SystemExit(1)

# Run Training Script and Save Output to Log
This cell runs the training script using subprocess, streams output to the notebook, and saves all lines to log.txt with timestamps.

In [ ]:
import subprocess

cmd = [
    "python", "-m", executable_path,
    "--f", dataset_dir,
    "--batch", batch,
    "--epochs", epochs,
    "--img_dir", img_dir
]
process = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

print("Running:", " ".join(cmd))

log_path = "log.txt"
with open(log_path, "a", encoding="utf-8") as log_f:
    log_f.write(f"\n--- Run started at {__import__('datetime').datetime.now().isoformat()} ---\n")
    process = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

    for line in process.stdout:
        print(line, end='')
        log_f.write(line)
        log_f.flush()

    returncode = process.wait()
    log_f.write(f"\n--- Run finished with exit code {returncode} at {__import__('datetime').datetime.now().isoformat()} ---\n")
    
process.wait()

# Graph Training Loss Evolution
This cell parses the log file for epoch loss lines and plots the training loss evolution using matplotlib.

In [ ]:
import re
from pathlib import Path
import matplotlib.pyplot as plt

log_path = Path("log.txt")
if not log_path.exists():
    raise FileNotFoundError(f"{log_path} not found. Run training first to create the log.")

pattern = re.compile(r"Epoch\s+(\d+)[^\d]*Loss:\s*([-+]?\d+(?:\.\d+)?(?:[eE][-+]?\d+)?)")
epoch_loss = {}

with log_path.open("r", encoding="utf-8") as f:
    for line in f:
        m = pattern.search(line)
        if m:
            epoch = int(m.group(1))
            loss = float(m.group(2))
            # keep last seen loss for the epoch (overwrites if duplicate)
            epoch_loss[epoch] = loss

if not epoch_loss:
    raise RuntimeError("No epoch loss lines found in log.txt. Expected lines like: 'Epoch 1, Loss: 0.1234'")

# sort by epoch and prepare lists
epochs = sorted(epoch_loss.keys())
losses = [epoch_loss[e] for e in epochs]

plt.figure(figsize=(8,5))
plt.plot(epochs, losses, marker='o', linestyle='-')
plt.title("Training Loss per Epoch (from log.txt)")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.grid(True)
plt.xticks(epochs)
plt.tight_layout()
plt.show()

# optionally save plot
plt.savefig("training_loss.png", dpi=150)
# ...existing code...

# (Optional) Custom Analysis or Next Steps
Use this cell for further analysis, saving results, or experimenting with outputs after training and visualization.